In [2]:
import tensorflow as tf
from keras.applications import InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

In [3]:
import cv2 

In [33]:
# Set the input image size
input_size = (512, 512)

In [34]:

# Load the Inception v3 model without the top classification layer
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(input_size[0], input_size[1], 3))


In [35]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False


In [36]:
# Add a new classification layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)  # 4 output units for the tumor classes


In [37]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [39]:

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

In [40]:
# Load and convert the images to grayscale before feeding them to the ImageDataGenerator
def load_and_convert_image(file_path):
    image = cv2.imread(file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, target_size)
    image = image.reshape((*target_size, 1))
    return image

In [41]:

test_datagen = ImageDataGenerator(rescale=1./255)
 
path = 'C:\\Users\\Abhijith_Kumble\\Desktop\\IEEE\\brain_Tumor_final\\brain_tumor'

# Set the paths to your training and testing data
train_data_dir = path+'\\Training'
test_data_dir = path + '\\Testing'

# Set the batch size
batch_size = 32

# Generate training and testing data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical',
    
    
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical',
    
)

# Set the number of training and testing samples
num_train_samples = train_generator.samples
num_test_samples = test_generator.samples

# Set the number of training epochs
num_epochs = 10



Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [42]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=num_epochs,
    validation_data=test_generator,
    validation_steps=num_test_samples // batch_size
)

Epoch 1/10
178/178 [==============================] - 1148s 6s/step - loss: 0.5721 - accuracy: 0.7866 - val_loss: 0.6913 - val_accuracy: 0.7859
Epoch 2/10
178/178 [==============================] - 1550s 9s/step - loss: 0.3579 - accuracy: 0.8681 - val_loss: 0.3805 - val_accuracy: 0.8625
Epoch 3/10
178/178 [==============================] - 1015s 6s/step - loss: 0.3317 - accuracy: 0.8808 - val_loss: 0.4094 - val_accuracy: 0.8383
Epoch 4/10
178/178 [==============================] - 1007s 6s/step - loss: 0.3209 - accuracy: 0.8843 - val_loss: 0.3349 - val_accuracy: 0.8828
Epoch 5/10
178/178 [==============================] - 1402s 8s/step - loss: 0.2849 - accuracy: 0.8915 - val_loss: 0.4076 - val_accuracy: 0.8500
Epoch 6/10
178/178 [==============================] - 1046s 6s/step - loss: 0.2848 - accuracy: 0.8917 - val_loss: 0.3202 - val_accuracy: 0.8742
Epoch 7/10
178/178 [==============================] - 953s 5s/step - loss: 0.2817 - accuracy: 0.8981 - val_loss: 0.4683 - val_accuracy: 

In [43]:
model.save('inception_v3.h5')

In [6]:
model = tf.keras.models.load_model('inception_v3.h5')

In [7]:
model.predict()

NameError: name 'test_generator' is not defined